<a href="https://colab.research.google.com/github/GurgelLucas/weather-cite/blob/main/spark_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio: Consumo de Dados para Previsão do Tempo das Cidades do Vale do Paraíba.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Neste desafio, você desenvolverá um notebook que será responsável por extrair dados de previsão do tempo das cidades do Vale do Paraíba, região onde se localiza a Dataside. Para consultar todas as cidades dessa região, utilizaremos a API do IBGE. No caso, basta realizar uma requisição HTTP com o método GET, utilizando a URL abaixo:

```
https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios
```

Com esses dados, gerar um data frame e a partir dele uma temp view. Ex: "cities"

Utilizando os nomes das cidades, deverão ser consultados os dados de previsão de tempo para cada cidade. Para realizar essa consulta, poderá ser utilizada qualquer uma das APIs informadas no link abaixo.

[Public APIs - Wather](https://github.com/public-apis/public-apis#weather)

Obs.: Para algumas, pode ser necessário cadastrar-se para acessar sua API Key. Mas nenhuma delas deve precisar cadastrar cartão de crédito ou adicionar qualquer valor monetário para utilizar. Caso alguma solicite, basta optar por outra.

Com os dados consultados, gerar um data frame e partir dele outra temp view. Ex: "forecasts"

Com as temp views geradas, utilizar Spark SQL para criar queries e gerar data frames das seguintes tabelas:

- Tabela 1: dados de previsão do tempo para os próximos cinco dias, para cada data e cidade consultadas. As colunas dessa tabela serão:
    - Cidade
    - CodigoDaCidade
    - Data
    - Regiao
    - Pais
    - Latitude
    - Longigute
    - TemperaturaMaxima
    - TemperaturaMinima
    - TemperaturaMedia
    - VaiChover
    - ChanceDeChuva
    - CondicaoDoTempo
    - NascerDoSol
    - PorDoSol
    - VelocidadeMaximaDoVento
    
    Obs.: Os valores da coluna "VaiChover" deverá ser "Sim" ou "Não". E a coluna "CodigoDaCidade" é o ID retornado junto com os nomes da cidades na API do IBGE.
    Obs.: Dependendo da API utilizada, algumas colunas podem não existir e ficarão em branco. Você deve optar por uma API que traga o maior número de informações possível.

- Tabela 2: quantidade de dias com chuva e sem chuva para os dias consultados, para cada data consultada. Colunas:
    - Cidade
    - QtdDiasVaiChover
    - QtdDiasNaoVaiChover
    - TotalDiasMapeados

Essas tabelas deverão ser exportadas em formado CSV e entregue no final do desafio.

## To Do

[ ] - Consultar municípios do Vale do Paraíba, gerar um data frame e criar uma temp view com esses dados.
[ ] - Consultar dados do tempo para cada município, gerar um data frame e criar uma outra temp view.
[ ] - Utilizar Spark SQL para gerar os data frames das Tabelas 1 e 2.
[ ] - Exportar os data frames para CSV.

## Atenção

- Existe um limite de requisições de 10000 requests por conta cadastrada na m3o.
- Essa API pode retornar cidades de outras regiões que possuem nome semelhante a alguma cidade do Vale do Paraiba. Pode mantê-las ou filtrar para gerar as tabelas apenas com dados de Regiao = Sao Paulo. Fica a seu critério.

## Entregando o desafio

Concluindo todos os passos informados em To Do, basta salvar o arquivo .ipynb do notebook e enviar para a Dataside juntamente com os CSVs das duas tabelas.


In [2]:
# instalar o JAVA
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# Baixar a versão mais recente do Spark
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [4]:
# Deszipar o spark
!tar xf /content/spark-3.1.2-bin-hadoop2.7.tgz
!rm spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
# Declarando as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [6]:
# Instalar a lib que ajuda e a encontrar o Spark no sistema e importá-lo como uma biblioteca regular
!pip install -q findspark unidecode

     |████████████████████████████████| 235 kB 13.9 MB/s 


In [8]:
import findspark
findspark.init()

import urllib.parse
import requests
import json
import unidecode
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col, explode_outer

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

In [21]:
# Buscar cidades do Vale do Paraíba
# TODO

req = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios")
data = req.json()

# Criar data frame com as cidades
# TODO

cities = spark.read.json(spark.sparkContext.parallelize([data]))
cities.printSchema()

# Flatten Data Cities 
df2 = cities.select(\
                    col("id"), \
                    col("microrregiao.id"), \
                    col("microrregiao.nome"), \
                    col("microrregiao.mesorregiao.id"), \
                    col("microrregiao.mesorregiao.nome"), \
                    col("microrregiao.mesorregiao.UF.id"), \
                    col("microrregiao.mesorregiao.UF.nome"), \
                    col("microrregiao.mesorregiao.UF.regiao.id"), \
                    col("microrregiao.mesorregiao.UF.regiao.nome"), \
                    col("microrregiao.mesorregiao.UF.regiao.sigla"), \
                    col("nome"), \
                    col("regiao-imediata.id"), \
                    col("regiao-imediata.nome"), \
                    col("regiao-imediata.regiao-intermediaria.id"), \
                    col("regiao-imediata.regiao-intermediaria.nome"), \
                  )
cities = df2.toDF(\
                      "id", \
                      "microregiao_id", \
                      "microregiao_nome", \
                      "mesorregiao_id", \
                      "mesorregiao_nome", \
                      "UF_id", \
                      "UF_nome", \
                      "regiao_id", \
                      "regiao_nome", \
                      "regiao_sigla", \
                      "nome", \
                      "regiao-imediata_id", \
                      "regiao-imediata_nome", \
                      "regiao-intermediaria_id", \
                      "regiao-intermediaria_nome", \
                    )

# Criar view com as cidades
# TODO
vw_cities = cities.createOrReplaceTempView("cities")
#spark.sql("SELECT microrregiao FROM cities").show()

root
 |-- id: string (nullable = true)
 |-- microrregiao: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- mesorregiao: struct (nullable = true)
 |    |    |-- UF: struct (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- nome: string (nullable = true)
 |    |    |    |-- regiao: struct (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- nome: string (nullable = true)
 |    |    |    |    |-- sigla: string (nullable = true)
 |    |    |    |-- sigla: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- nome: string (nullable = true)
 |    |-- nome: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- regiao-imediata: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- regiao-intermediaria: struct (nullable = true)
 |    |    |-- UF: struct (nullable = true)
 |    |    |    |-- id: long 

In [65]:
# Buscar previsão do tempo para as cidades
# TODO

data = []
for city in cities.collect():
  cidade = urllib.parse.quote(city.nome)
  req = requests.get(f"http://api.weatherapi.com/v1/forecast.json?key=49e9a2605cc34c618b172338220405&q={cidade}&days=5&aqi=no&alerts=no")
  res = req.json()
  data.append(res)

# Criar data frame com as previsões
# TODO
weather_forecast = spark.read.json(spark.sparkContext.parallelize(data))
weather_forecast = weather_forecast.filter(weather_forecast.location.region == "Sao Paulo")
weather_forecast.printSchema()

# Criar view com as previsões
# TODO
vw_weather_forecast = weather_forecast.createOrReplaceTempView("weather_forecast")

root
 |-- current: struct (nullable = true)
 |    |-- cloud: long (nullable = true)
 |    |-- condition: struct (nullable = true)
 |    |    |-- code: long (nullable = true)
 |    |    |-- icon: string (nullable = true)
 |    |    |-- text: string (nullable = true)
 |    |-- feelslike_c: double (nullable = true)
 |    |-- feelslike_f: double (nullable = true)
 |    |-- gust_kph: double (nullable = true)
 |    |-- gust_mph: double (nullable = true)
 |    |-- humidity: long (nullable = true)
 |    |-- is_day: long (nullable = true)
 |    |-- last_updated: string (nullable = true)
 |    |-- last_updated_epoch: long (nullable = true)
 |    |-- precip_in: double (nullable = true)
 |    |-- precip_mm: double (nullable = true)
 |    |-- pressure_in: double (nullable = true)
 |    |-- pressure_mb: double (nullable = true)
 |    |-- temp_c: double (nullable = true)
 |    |-- temp_f: double (nullable = true)
 |    |-- uv: double (nullable = true)
 |    |-- vis_km: double (nullable = true)
 |    

In [73]:
# Criar DF da Tabela 1
# TODO
#spark.sql("SELECT c.*, wf.* FROM cities as c LEFT JOIN weather_forecast as wf ON c.nome = wf.location.name").show()
spark.sql("\
SELECT \
  c.nome as Cidade, \
  c.id as CodigoDaCidade, \
  wf.forecast.forecastday.date as Data, \
  c.regiao_nome as Regiao, \
  wf.location.country as Pais, \
  wf.location.lat as Latitude, \
  wf.location.lon as Longitude, \
  wf.forecast.forecastday.day.mintemp_c as TemperaturaMinima, \
  wf.forecast.forecastday.day.maxtemp_c as TemperaturaMaxima, \
  wf.forecast.forecastday.day.avgtemp_c as TemperaturaMedia, \
  wf.forecast.forecastday.day.daily_will_it_rain as VaiChover, \
  wf.forecast.forecastday.day.daily_chance_of_rain as ChanceDeChuva, \
  wf.forecast.forecastday.astro.sunrise as NascerDoSol, \
  wf.forecast.forecastday.astro.sunset as PorDoSol, \
  wf.forecast.forecastday.day.maxwind_kph as VelocidadeMaximaDoVento \
FROM \
  cities as c \
LEFT JOIN \
  weather_forecast as wf \
ON \
  c.nome = wf.location.name"
).show()

+--------------------+--------------+--------------------+-------+------+--------+---------+------------------+------------------+------------------+---------+-------------+--------------------+--------------------+-----------------------+
|              Cidade|CodigoDaCidade|                Data| Regiao|  Pais|Latitude|Longitude| TemperaturaMinima| TemperaturaMaxima|  TemperaturaMedia|VaiChover|ChanceDeChuva|         NascerDoSol|            PorDoSol|VelocidadeMaximaDoVento|
+--------------------+--------------+--------------------+-------+------+--------+---------+------------------+------------------+------------------+---------+-------------+--------------------+--------------------+-----------------------+
|São Bento do Sapucaí|       3548609|                null|Sudeste|  null|    null|     null|              null|              null|              null|     null|         null|                null|                null|                   null|
|           Paraibuna|       3535606|[20

In [ ]:
# Criar DF da Tabela 2
# TODO

In [ ]:
# Exportar CSVs
# TODO